# Import Required Libraries
Import the necessary libraries, including yfinance and pandas.

In [1]:
!pip install yfinance pandas
!pip install --upgrade 'jinja2>=3.1.2'

import yfinance as yf
import pandas as pd
import math

# Define Stock Symbols
Define a list of stock symbols to watch, including MFC.TO, RY.TO, ENB.TO, EQB.TO, PRL.TO, MAL.TO, DRX.TO, AIDX.TO, MSFT, GOOG, AAPL, NVDA, AMD, WELL.TO, BTB-UN.TO, JPM.NE, BIP-UN.TO.

In [7]:
# Define a list of stock symbols to watch
stock_symbols = [
    ["MFC.TO", 1], ["RY.TO", 1], ["ENB.TO", 1], ["EQB.TO",1], ["PRL.TO",1], ["MAL.TO",1], ["SURG.V",1], ["DRX.TO",1], ["WELL.TO",1], ["JPM.NE",1], 
    ["AIDX.TO",1], ["AMZN",1], ["MSFT",1], ["GOOG",1], ["AAPL",1], ["NVDA",1], ["FTAI",1], ["BTB-UN.TO",1], ["WSP.TO",1], ["ORE.TO",1], ["COST",1], ["WMT", 1],
    ["ALV.V",1], ["MND.TO",1], ["PSD.TO",1], ["FDI.V",1], ["TAL.TO",1], ["FAR.TO",1], ["CTZ.V",1], ["EW.V",1], ["SVM.TO",1], ["WINS.V",1],
    ["SYF",1], ["CEU.TO",1], ["BDT.TO",1], ["FTG.TO",1], ["DNG.TO",1], ["CAF.V",1], ["IAG.TO",1], ["TCBX",1], ["QFIN",1]
]

# Fetch Stock Data
Use yfinance to fetch the current or last closing price, gains or loss for the current or last trading day, and stock analyst price targets for 3 months high and low for each stock.

In [8]:
def calc_targets(last_close, stock_info):
    percent = lambda a, b: ((a - b)/b) * 100
    price_target_mean = stock_info.get('targetMeanPrice', None)
    price_target_high = stock_info.get('targetHighPrice', None)
    price_target_low = stock_info.get('targetLowPrice', None)
    if not math.isnan(price_target_mean):
        return [price_target_mean, percen(price_target_mean, last_close),
                price_target_high, percen(price_target_high, last_close),
                price_target_low, percen(price_target_low, last_close)]
    else:
        return [0, 0,
                0, 0,
                0, 0]

# Function to fetch stock data
def fetch_stock_data(symbols):
    stock_data = []
    for symbol in symbols:
        stock = yf.Ticker(symbol[0])
        hist = stock.history(period="5d")
        if not hist.empty:
            last_close = hist['Close'].iloc[-1]
            prev_close = hist['Close'].iloc[-2] if len(hist) > 1 else last_close
            gain_loss = last_close - prev_close
            price_target_mean = stock.info.get('targetMeanPrice', None)
            price_target_high = stock.info.get('targetHighPrice', None)
            price_target_low = stock.info.get('targetLowPrice', None)
            recommendations = stock.recommendations
            if not recommendations.empty:
                strong_buy = recommendations.get('strongBuy')[0]
                buy = recommendations.get('buy')[0]
                hold = recommendations.get('hold')[0]
                sell = recommendations.get('sell')[0]
                strong_sell = recommendations.get('strongSell')[0]
                maen_percent = (price_target_mean - last_close)/last_close
                high_percent = (price_target_high - last_close)/last_close
                low_percent = (price_target_low - last_close)/last_close
                high_target_growth = 1 + high_percent
                maen_target_growth = 1 + maen_percent
                low_target_growth = 1 + low_percent
                low_gains = (last_close * low_target_growth * symbol[1])
                mean_gains = (last_close * maen_target_growth * symbol[1]) 
                high_gains = (last_close * high_target_growth * symbol[1])
                tmpercent = maen_percent * 100
                thpercent = high_percent * 100
                tlpercent = low_percent * 100
                
            else:
                strong_buy = 0
                buy = 0
                hold = 0
                sell = 0
                strong_sell = 0
                price_target_mean = 0
                tmpercent = 0
                price_target_high = 0
                thpercent = 0
                price_target_low = 0
                tlpercent = 0
                mean_gains = 0
                
            # Determine the color based on gain/loss
            color = 'green' if gain_loss > 0 else 'red'
            
            stock_data.append({
                'Symbol': symbol,
                'LastClose': f"{last_close:.2f}",
                'GainLoss': f"{gain_loss:.2f}",
                'TLG': f"{low_gains:.2f}",
                'TMG': f"${mean_gains:.2f}",
                'THG': f"${high_gains:.2f}",
                'StrongBuy': strong_buy,
                'Buy': buy,
                'Hold': hold,
                'Sell': sell,
                'StrongSell': strong_sell,
                'Color': color
            })
    return pd.DataFrame(stock_data)

# Fetch stock data for the defined symbols
stock_df = fetch_stock_data(stock_symbols)
stock_df


,Symbol,LastClose,GainLoss,TLG,TMG,THG,StrongBuy,Buy,Hold,Sell,StrongSell,Color
0,"[MFC.TO, 1]",44.26,0.19,34.00,$46.57,$50.00,6,3,4,0,1,green
1,"[RY.TO, 1]",177.39,0.08,139.00,$185.71,$200.00,6,6,3,0,1,green
2,"[ENB.TO, 1]",59.23,-0.46,53.00,$61.10,$72.00,3,6,12,1,0,red
3,"[EQB.TO, 1]",99.52,0.01,109.00,$117.14,$130.00,3,1,3,0,0,green
4,"[PRL.TO, 1]",37.12,0.25,42.49,$45.11,$47.65,3,4,0,0,0,green
5,"[MAL.TO, 1]",10.35,-0.23,12.00,$15.00,$18.00,0,2,0,0,0,red
6,"[SURG.V, 1]",0.09,-0.01,0.30,$0.30,$0.30,1,0,0,0,0,red
7,"[DRX.TO, 1]",10.38,0.68,23.00,$23.00,$23.00,0,1,0,0,0,green
8,"[WELL.TO, 1]",6.99,0.20,7.00,$8.50,$11.00,6,8,1,0,0,green
9,"[JPM.NE, 1]",31.90,-0.02,7.00,$0.00,$11.00,0,0,0,0,0,red
